In [ ]:
import pandas as pd
import sqlite3

In [ ]:
df = pd.read_csv('EASminiprojectDataset.csv')

# Introduction:
  Database normalization is a process of organizing data in a database to reduce redundancy and improve data integrity. It involves arranging data into tables and establishing relationships between them according to specific rules, known as normal forms. The goal is to create a database that is efficient, consistent, and easier to manage.

 Given the data set below contained data about the factors that would lead to a person sleeping disorder. Our goal is to normalize the data set to 3rd normal form to help the data clean easy to maintain. Since every column in the data set hold atomic values, a primary key is defined as PersonId, and each cell holds a single value and rows are uniquely identified by PersonID hence it is already in 1st normal form. Since the data set only have the PersonId as primary key, we don't have to worry about partial dependencies on primary keys, hence we will focus on removing repeating values into seperate tables or redundant data that can be stored seperately. Here we can seperate Occupation and Sleep Disorder into 2 different tables and store them as an ID in the Person table so the data set now in 2NF. Now the table is in 2NF and also 3NF since there is not transitive dependencies so the data set is very well structured and easier to maintain.

 # So we will have the following tables:
  ## Person Table:
     PersonID INTEGER PRIMARY KEY,
     Gender TEXT NOT NULL,
     Age INTEGER NOT NULL,
     OccupationID INTEGER NOT NULL,
     SleepDuration REAL NOT NULL,
     QualityOfSleep INTEGER NOT NULL,
     PhysicalActivityLevel INTEGER NOT NULL,
     StressLevel INTEGER NOT NULL,
     BMICategoryID INTEGER NOT NULL,
     BloodPressureID TEXT NOT NULL,
     HeartRate INTEGER NOT NULL,
     DailySteps INTEGER NOT NULL,
     SleepDisorderID INTEGER

   Occupation Table:

     OccupationID INTEGER PRIMARY KEY,
     Occupation TEXT NOT NULL

   SleepDisorder Table:

     SleepDisorderID INTEGER PRIMARY KEY,
     SleepDisorder TEXT NOT NULL
   BMICategory Table:

     BMICategoryID INTEGER PRIMARY KEY,
     BMICategory TEXT NOT NULL

In [ ]:
# Check column name
print(df.columns.tolist())

['Person ID', 'Gender', 'Age', 'Occupation', 'Sleep Duration', 'Quality of Sleep', 'Physical Activity Level', 'Stress Level', 'BMI Category', 'Blood Pressure', 'Heart Rate', 'Daily Steps', 'Sleep Disorder']


In [ ]:
# Rename columns to match table
df.rename(columns={
    'Person ID': 'PersonID',
    'Sleep Duration': 'SleepDuration',
    'Quality of Sleep': 'QualityOfSleep',
    'Physical Activity Level': 'PhysicalActivityLevel',
    'Stress Level': 'StressLevel',
    'BMI Category': 'BMICategory',
    'Blood Pressure': 'BloodPressureID',
    'Heart Rate': 'HeartRate',
    'Daily Steps': 'DailySteps',
    'Sleep Disorder': 'SleepDisorder'
}, inplace=True)

# Replace NaN with None
df = df.where(pd.notnull(df), None)

# Create lookup tables
# Basically, we create a dictionary that map all the unique, non-null in the columns to the unique ID so later on we can use it to create a new table and use the ID to map it in
# the person table
occupation_lookup = {name: idx + 1 for idx, name in enumerate(df['Occupation'].dropna().unique())}
bmi_lookup = {name: idx + 1 for idx, name in enumerate(df['BMICategory'].dropna().unique())}
sleep_disorder_lookup = {name: idx + 1 for idx, name in enumerate(df['SleepDisorder'].dropna().unique())}
sleep_disorder_lookup[None] = None  # For NaN entries

# Map values to IDs
# Basically, we creates a new column 'Col' by replacing each 'value' with its corresponding ID using the 'look_up' dictionary that created above.
df['OccupationID'] = df['Occupation'].map(occupation_lookup)
df['BMICategoryID'] = df['BMICategory'].map(bmi_lookup)
df['SleepDisorderID'] = df['SleepDisorder'].map(sleep_disorder_lookup)


# Select columns for Person table
person_df = df[[
    'PersonID', 'Gender', 'Age', 'OccupationID', 'SleepDuration', 'QualityOfSleep',
    'PhysicalActivityLevel', 'StressLevel', 'BMICategoryID', 'BloodPressureID',
    'HeartRate', 'DailySteps', 'SleepDisorderID'
]]

# Create SQLite DB
conn = sqlite3.connect("health_data.db")
cursor = conn.cursor()

# Create reference tables
cursor.execute('''CREATE TABLE IF NOT EXISTS Occupation (
    OccupationID INTEGER PRIMARY KEY,
    Occupation TEXT NOT NULL
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS BMICategory (
    BMICategoryID INTEGER PRIMARY KEY,
    BMICategory TEXT NOT NULL
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS SleepDisorder (
    SleepDisorderID INTEGER PRIMARY KEY,
    SleepDisorder TEXT
)''')

# Create main Person table
cursor.execute('''CREATE TABLE IF NOT EXISTS Person (
    PersonID INTEGER PRIMARY KEY,
    Gender TEXT NOT NULL,
    Age INTEGER NOT NULL,
    OccupationID INTEGER NOT NULL,
    SleepDuration REAL NOT NULL,
    QualityOfSleep INTEGER NOT NULL,
    PhysicalActivityLevel INTEGER NOT NULL,
    StressLevel INTEGER NOT NULL,
    BMICategoryID INTEGER NOT NULL,
    BloodPressureID TEXT NOT NULL,
    HeartRate INTEGER NOT NULL,
    DailySteps INTEGER NOT NULL,
    SleepDisorderID INTEGER,
    FOREIGN KEY (OccupationID) REFERENCES Occupation(OccupationID),
    FOREIGN KEY (BMICategoryID) REFERENCES BMICategory(BMICategoryID),
    FOREIGN KEY (SleepDisorderID) REFERENCES SleepDisorder(SleepDisorderID)
)''')

# Insert lookup values
cursor.executemany("INSERT INTO Occupation VALUES (?, ?)", [(v, k) for k, v in occupation_lookup.items()])
cursor.executemany("INSERT INTO BMICategory VALUES (?, ?)", [(v, k) for k, v in bmi_lookup.items()])
cursor.executemany("INSERT INTO SleepDisorder VALUES (?, ?)", [(v, k) for k, v in sleep_disorder_lookup.items() if k is not None])

# Insert main data
person_df.to_sql('Person', conn, if_exists='append', index=False)

conn.commit()
conn.close()

In [ ]:
                                                                  # Average Sleep Duration by Occupation
conn = sqlite3.connect('health_data.db')

# Load a specific table (e.g., Person)
sql = '''
SELECT
    Occupation.Occupation,
    ROUND(AVG(Person.SleepDuration), 2) AS AvgSleepDuration
FROM
    Person
JOIN
    Occupation ON Person.OccupationID = Occupation.OccupationID
GROUP BY
    Occupation.Occupation
ORDER BY
    AvgSleepDuration DESC;

'''
df = pd.read_sql_query(sql, conn)
conn.close()
df

,Occupation,AvgSleepDuration
0,Engineer,7.99
1,Lawyer,7.41
2,Accountant,7.11
3,Nurse,7.06
4,Doctor,6.97
5,Manager,6.90
6,Software Engineer,6.75
7,Teacher,6.69
8,Salesperson,6.40
9,Scientist,6.00


In [ ]:
# Count of People by Sleep Disorder Type with different Roles
conn = sqlite3.connect('health_data.db')
sql = '''
SELECT
    SleepDisorder.SleepDisorder,
    Occupation.Occupation,
    COUNT(*) AS Count
FROM
    Person
JOIN
    SleepDisorder ON Person.SleepDisorderID = SleepDisorder.SleepDisorderID
JOIN
    Occupation ON Person.OccupationID = Occupation.OccupationID
GROUP BY
    SleepDisorder.SleepDisorder,
    Occupation.Occupation
ORDER BY
    Count DESC;
'''
df = pd.read_sql_query(sql, conn)
conn.close()
df

,SleepDisorder,Occupation,Count
0,Sleep Apnea,Nurse,61
1,Insomnia,Salesperson,29
2,Insomnia,Teacher,27
3,Insomnia,Accountant,7
4,Insomnia,Engineer,5
5,Sleep Apnea,Doctor,4
6,Sleep Apnea,Teacher,4
7,Insomnia,Doctor,3
8,Insomnia,Nurse,3
9,Sleep Apnea,Lawyer,3


In [ ]:
# Average Daily Steps for Obese/Normal Individuals
conn = sqlite3.connect('health_data.db')
sql = '''
SELECT
    ROUND(AVG(CASE WHEN BMICategory.BMICategory = 'Obese' THEN Person.DailySteps END), 2) AS Avg_Daily_Steps_For_Obese,
    ROUND(AVG(CASE WHEN BMICategory.BMICategory = 'Obese' THEN Person.SleepDuration END), 2) AS Avg_Sleep_Duration_For_Obese,
    ROUND(AVG(CASE WHEN BMICategory.BMICategory = 'Normal' THEN Person.DailySteps END), 2) AS Avg_Daily_Steps_For_Normal_Person,
    ROUND(AVG(CASE WHEN BMICategory.BMICategory = 'Normal' THEN Person.SleepDuration END), 2) AS Avg_Sleep_Duration_For_Normal_Person
FROM
    Person
JOIN
    BMICategory ON Person.BMICategoryID = BMICategory.BMICategoryID;
'''
df = pd.read_sql_query(sql, conn)
conn.close()
df

,Avg_Daily_Steps_For_Obese,Avg_Sleep_Duration_For_Obese,Avg_Daily_Steps_For_Normal_Person,Avg_Sleep_Duration_For_Normal_Person
0,3350.0,6.96,6887.18,7.39


In [ ]:
# Average Stress Level and Sleep Quality by Gender
conn = sqlite3.connect('health_data.db')
sql = '''
SELECT
    Gender,
    ROUND(AVG(StressLevel), 2) AS AvgStressLevel,
    ROUND(AVG(QualityOfSleep), 2) AS AvgSleepQuality
FROM
    Person
GROUP BY
    Gender;

'''
df = pd.read_sql_query(sql, conn)
conn.close()
df

,Gender,AvgStressLevel,AvgSleepQuality
0,Female,4.68,7.66
1,Male,6.08,6.97


In [ ]:
# Occupation with the Highest Average Stress Level
conn = sqlite3.connect('health_data.db')
sql = '''
SELECT
    Occupation.Occupation,
    ROUND(AVG(Person.StressLevel), 2) AS AvgStressLevel
FROM
    Person
JOIN
    Occupation ON Person.OccupationID = Occupation.OccupationID
GROUP BY
    Occupation.Occupation
ORDER BY
    AvgStressLevel DESC
;

'''
df = pd.read_sql_query(sql, conn)
conn.close()
df

,Occupation,AvgStressLevel
0,Sales Representative,8.00
1,Scientist,7.00
2,Salesperson,7.00
3,Doctor,6.73
4,Software Engineer,6.00
5,Nurse,5.55
6,Lawyer,5.06
7,Manager,5.00
8,Accountant,4.59
9,Teacher,4.53


In [ ]:
conn = sqlite3.connect('health_data.db')
sql = 'Select * from BMICategory'
sql1 = 'Select * from Person'
sql2 = 'Select * from Occupation'
sql3 = 'Select * from SleepDisorder'
df = pd.read_sql_query(sql3, conn)
conn.close()
df

,SleepDisorderID,SleepDisorder
0,1,Sleep Apnea
1,2,Insomnia
